# Guia de Estudo: Compreendendo o Código PSPNet

## Questionário
1) Qual é a função do arquivo data_utils.py no contexto do projeto PSPNet?
2) Descreva o propósito do arquivo inference_sample.py e como ele se relaciona com o modelo PSPNet treinado.
3) Explique a estrutura e funcionalidade da rede PSPNet implementada em psp_net.py.
4) Quais são os principais argumentos e opções de configuração disponíveis no script train_psp.py?
5) Como o processo de treinamento é realizado usando o script train_psp.py?
6) Descreva a importância do módulo Pyramid Pooling Module na arquitetura PSPNet.
7) Que tipos de dados de entrada e saída são esperados pelos scripts inference_sample.py e train_psp.py?
8) Com quais outras bibliotecas ou frameworks o código PSPNet provavelmente interage e por quê?
9) Quais métricas de avaliação você usaria para medir o desempenho do modelo PSPNet?
10) Como o código pode ser modificado para incorporar novos conjuntos de dados ou diferentes tarefas de segmentação semântica?

## Gabarito
- O arquivo data_utils.py contém funções utilitárias para lidar com dados, como carregamento, pré-processamento e aumento de dados para o treinamento do modelo PSPNet.
- O arquivo inference_sample.py permite realizar inferência com um modelo PSPNet treinado. Ele carrega o modelo, processa uma imagem de entrada e gera uma previsão de segmentação semântica.
- O arquivo psp_net.py define a arquitetura da rede PSPNet, incluindo a estrutura principal da rede, o módulo Pyramid Pooling e as camadas de saída. Ele implementa a lógica para propagação direta e cálculo de perdas.
- O script train_psp.py possui argumentos para configurar o processo de treinamento, como a taxa de aprendizado, o tamanho do lote, o número de épocas, o caminho para os dados de treinamento e o modelo pré-treinado.
- O script train_psp.py carrega os dados de treinamento, instancia o modelo PSPNet, define o otimizador e a função de perda, e executa o loop de treinamento, atualizando os pesos do modelo a cada época.
- O Pyramid Pooling Module captura informações contextuais de múltiplas escalas, agregando recursos em diferentes níveis de resolução, o que melhora a capacidade do modelo de segmentar objetos de tamanhos variados.
- Os scripts inference_sample.py e train_psp.py esperam imagens como entrada e produzem mapas de segmentação semântica como saída, onde cada pixel é atribuído a uma classe.
- O código PSPNet provavelmente interage com bibliotecas como TensorFlow ou PyTorch para operações de tensores e treinamento de redes neurais, OpenCV para processamento de imagens e NumPy para manipulação de matrizes.
- Métricas como precisão de pixel, Intersection over Union (IoU) e Dice coefficient são usadas para avaliar o desempenho de modelos de segmentação semântica como o PSPNet.
- Para incorporar novos conjuntos de dados ou tarefas, o código pode ser modificado ajustando as funções de carregamento de dados em data_utils.py, definindo novas classes de saída em psp_net.py e adaptando a lógica de treinamento em train_psp.py.

## Questões Dissertativas
- Compare e contraste a arquitetura PSPNet com outras arquiteturas de segmentação semântica, como FCN e U-Net. Destaque as principais diferenças em suas estruturas e como elas afetam o desempenho.
- Explique em detalhes como o Pyramid Pooling Module na PSPNet contribui para capturar informações contextuais multi-escala. Discuta os benefícios de usar esse módulo em tarefas de segmentação semântica.
- Analise o processo de treinamento do PSPNet implementado em train_psp.py. Descreva os diferentes estágios do treinamento, os hiperparâmetros envolvidos e as estratégias para otimizar o desempenho do modelo.
- Discuta os desafios e limitações do modelo PSPNet em cenários do mundo real. Explore as áreas potenciais para melhorias e como o código pode ser adaptado para lidar com esses desafios.
- Investigue aplicações práticas da segmentação semântica usando PSPNet em domínios como condução autônoma, análise de imagens médicas ou detecção de objetos. Forneça exemplos específicos e discuta o impacto da PSPNet nesses campos.

## Glossário
- Termo Definição Segmentação: Semântica Tarefa de visão computacional que visa atribuir um rótulo de classe a cada pixel em uma imagem.
- PSPNet: Arquitetura de rede neural profunda para segmentação semântica, conhecida por sua capacidade de capturar informações contextuais multi-escala.
- Pyramid Pooling Module: Módulo chave na PSPNet que agrega recursos em diferentes níveis de resolução para obter representações contextuais.
- Inferência: Processo de usar um modelo treinado para fazer previsões sobre novos dados.
- Treinamento: Processo de ajustar os pesos de um modelo neural para minimizar a diferença entre as previsões e os rótulos reais.
- Época: Uma passagem completa por todo o conjunto de dados de treinamento durante o processo de treinamento.
- Taxa de Aprendizado: Hiperparâmetro que controla o tamanho da etapa na atualização dos pesos do modelo durante o treinamento.
- Tamanho do Lote: Número de amostras de dados processadas em uma única iteração durante o treinamento.
- IoU (Intersection over Union): Métrica de avaliação para segmentação semântica que mede a sobreposição entre a previsão e o rótulo real.
- Precisão de Pixel: Métrica de avaliação que calcula a porcentagem de pixels corretamente classificados.

# Instalação de Pacotes

In [ ]:
%pip install numpy
%pip install matplotlib
%pip install torch
%pip install torchvision
%pip install utils2
%pip install opencv-python

# Bibliotecas Importadas

In [19]:
import os
import os.path as osp
import numpy as np
import utils2
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

import torch
import torchvision
from torch.utils.data import Dataset
import json
import cv2
import glob


from torch.utils.data import DataLoader
from math import sqrt
from torch.autograd import Variable

# Declaração de Variáveis

In [ ]:
# CUDA:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Caminho do diretório Dataset:
directory = os.path.abspath(os.path.join(os.getcwd(), '..')) + r'\Dataset\nematode-detection-labels'
print(f'Diretório do Projeto {directory}.')
if not os.path.exists(directory):
    os.makedirs(directory)
img_folder_val = directory + r'\Val'
img_folder_train = directory + r'\Train'
img_folder_test = directory + r'\Test'
save_dir = directory + r'\\result_psp\\'
if not os.path.exists(img_folder_val):
    os.makedirs(img_folder_val)
if not os.path.exists(img_folder_train):
    os.makedirs(img_folder_train)
if not os.path.exists(img_folder_test):
    os.makedirs(img_folder_test)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

model_file_name = 'model_psp.pth'

# size mismatch (got input: [1, 3, 480, 640] , target: [1, 960, 1280]
resolution_input = (640, 480) # Size Out (1280, 960)

patience = 30
plot_val = True
plot_train = True

max_epochs = 100

class_weights = [1, 1, 1]
nClasses = 3

class_to_color = {'Ground': (127, 0, 0) , 'Healthy': (0, 127, 127), 'Pest': (0, 255, 0)}
class_to_id = {'Ground': 0, 'Healthy': 1, 'Pest': 2}
id_to_class = {v: k for k, v in class_to_id.items()}

# Define a classe de Segmentation do Dataset

In [21]:
class SegmentationDataset(Dataset):
    """Segmentation dataset loader."""

    def __init__(self, json_folder, img_folder, is_train, class_to_id, resolution_input = (640, 480), augmentation = False, transform=None):
    #def __init__(self, json_folder, img_folder, is_train, class_to_id, resolution_input = (1280, 960), augmentation = False, transform=None):
        """
        Args:
            json_folder (str): Path to folder that contains the annotations.
            img_folder (str): Path to all images.
            is_train (bool): Is this a training dataset ?
            augmentation (bool): Do dataset augmentation (crete artificial variance) ?
        """

        self.gt_file_list = glob.glob(osp.join(json_folder, '*.json'))

        self.total_samples = len(self.gt_file_list)
        self.img_folder = img_folder
        self.is_train = is_train
        self.transform = transform
        self.augmentation = augmentation
        self.resolution = resolution_input
        self.class_to_id = class_to_id
        
        
        # Mean and std are needed because we start from a pre trained net
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]

    def __len__(self):
        return self.total_samples

    def __getitem__(self, idx):
        
        gt_file = self.gt_file_list[idx]
        img_number_str = gt_file.split('.')[0].split('/')[-1]
	# Abre Json
        gt_json = json.load(open(gt_file, 'r'))
	# Abre imagem
        img_np = cv2.imread(osp.join(self.img_folder, img_number_str + '.png'), cv2.IMREAD_IGNORE_ORIENTATION + cv2.IMREAD_COLOR)
        original_shape = img_np.shape
        img_np = cv2.resize(img_np, (self.resolution[0], self.resolution[1]))[..., ::-1]
        img_np = np.ascontiguousarray(img_np)
	# Cria imagem zerada
        label_np = np.zeros((img_np.shape[0], img_np.shape[1]))
        label_np[...] = -1
        
	# Para todos poligonos
        for shape in gt_json['shapes']:
            # Transforma os pontos do poligono em array
            points_np = np.array(shape['points'], dtype = np.float64)

	    # Ajusta os pontos porque eu mudo o resolucao (pode ignorar)
            points_np[:, 0] *= self.resolution[0]/original_shape[1]
            points_np[:, 1] *= self.resolution[1]/original_shape[0]
	    # As coordenadas dos pontos que formam o poligono tem que ser inteiros
            points_np = np.round(points_np).astype(np.int64)
	    # Coloca os pontos no formato certo para o opencv
            points_np = points_np.reshape((-1,1,2))
	    # Pinta o poligono usando o opencv com o valor referente ao label
            label_np = cv2.fillPoly(label_np, [points_np], self.class_to_id[shape['label']])

        # Transforma o GT em inteiro    
        label_np = label_np.astype(np.int32)
        
        if self.is_train and self.augmentation:
            if np.random.rand() > 0.5:
                img_np = np.fliplr(img_np)
                label_np = np.fliplr(label_np)
                img_np = np.ascontiguousarray(img_np)
                label_np = np.ascontiguousarray(label_np)
        
        img_pt = img_np.astype(np.float32) / 255.0
        for i in range(3):
            img_pt[..., i] -= self.mean[i]
            img_pt[..., i] /= self.std[i]
            
        img_pt = img_pt.transpose(2,0,1)
            
        img_pt = torch.from_numpy(img_pt)
        label_pt = torch.from_numpy(label_np).long()
        #print(img_number_str, img_pt.shape)

        sample = {'image': img_pt, 'gt': label_pt, 'image_original': img_np}

        if self.transform:
            sample = self.transform(sample)

        return sample

# Define a Classe da Rede PSPDec

In [22]:
class PSPDec(torch.nn.Module):
	def __init__(self, in_dim, reduction_dim, setting):
		super(PSPDec, self).__init__()
		self.features = []
		for s in setting:
			self.features.append(torch.nn.Sequential(
				torch.nn.AdaptiveAvgPool2d(s),
				torch.nn.Conv2d(in_dim, reduction_dim, kernel_size=1, bias=False),
				#torch.nn.BatchNorm2d(reduction_dim, momentum=.95),
				#torch.nn.InstanceNorm2d(reduction_dim, momentum=.95),
				torch.nn.ReLU(inplace=True)
			))
		self.features = torch.nn.ModuleList(self.features)


	def forward(self, x):
		x_size = x.size()
		out = [x]
		for f in self.features:
			out.append(torch.nn.functional.upsample(f(x), x_size[2:], mode='bilinear'))
		out = torch.cat(out, 1)
		return out

# Define a Classe da Rede PSPNet

In [23]:
class PSPNet(torch.nn.Module):

	def __init__(self, num_classes):
		super(PSPNet, self).__init__()

		resnet = torchvision.models.resnet101(pretrained=True)
		#print('resnet', resnet)

		self.layer0 = torch.nn.Sequential(
			resnet.conv1,
			resnet.bn1,
			resnet.relu,
			resnet.maxpool
		)
		
		self.layer1 = resnet.layer1
		self.layer2 = resnet.layer2
		self.layer3 = resnet.layer3
		self.layer4 = resnet.layer4
		

		for n, m in self.layer3.named_modules():
			if 'conv2' in n:
				m.dilation, m.padding, m.stride = (2, 2), (2, 2), (1, 1)
			elif 'downsample.0' in n:
				m.stride = (1, 1)
		for n, m in self.layer4.named_modules():
			if 'conv2' in n:
				m.dilation, m.padding, m.stride = (4, 4), (4, 4), (1, 1)
			elif 'downsample.0' in n:
				m.stride = (1, 1)


		self.ppm = PSPDec(2048, 512, (1, 2, 3, 6))

		self.final = torch.nn.Sequential(
			torch.nn.Conv2d(4096, 512, 3, padding=1, bias=False),
			torch.nn.BatchNorm2d(512, momentum=.95),
			torch.nn.ReLU(inplace=True),
			torch.nn.Dropout(.1),
			torch.nn.Conv2d(512, num_classes, 1),
		)


	def forward(self, x):
		x = self.layer0(x)
		x = self.layer1(x)
		x = self.layer2(x)
		x = self.layer3(x)
		x = self.layer4(x)
		
		x = self.ppm(x)
		x = self.final(x)
		return torch.nn.functional.upsample(x, (480, 640), mode='bilinear')


	@staticmethod
	def eval_net_with_loss(model, inp, gt, class_weights, device):

		weights = torch.from_numpy(np.array(class_weights, dtype=np.float32)).to(device)
		out = model(inp)

		softmax = torch.nn.functional.log_softmax(out, dim = 1)
		loss = torch.nn.functional.nll_loss(softmax, gt, ignore_index=-1, weight=weights)

		return (out, loss)

# Realiza o Treinamento da Rede

In [ ]:
train_dataset = SegmentationDataset(img_folder_train, img_folder_train, True, class_to_id, resolution_input, True)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=0, drop_last=False)

val_dataset = SegmentationDataset(img_folder_val, img_folder_val, False, class_to_id, resolution_input)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True, num_workers=0, drop_last=False)


if plot_train:

    for i_batch, sample_batched in enumerate(train_loader):
    
            image_np = np.squeeze(sample_batched['image_original'].cpu().numpy())
            gt = np.squeeze(sample_batched['gt'].cpu().numpy())
                
            color_label = np.zeros((resolution_input[1], resolution_input[0], 3))
            
            for key, val in id_to_class.items():
                color_label[gt == key] = class_to_color[val]
                
            plt.figure()
            plt.imshow((image_np/255) * 0.5 + (color_label/255) * 0.5)
            plt.show()
            
            plt.figure()
            plt.imshow(color_label.astype(np.uint8))
            plt.show()

model = PSPNet(nClasses).to(device)

optimizer = torch.optim.SGD(model.parameters(), 1e-2, .9, 1e-4)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 30, gamma=0.2)

best_val_acc = -1
best_epoch = 0

# Start training...
for epoch in range(max_epochs):
    print('Epoch %d starting...' % (epoch+1))

    lr_scheduler.step()
    model.train()

    mean_loss = 0.0
    n_correct = 0
    n_false = 0

    for i_batch, sample_batched in enumerate(train_loader):
        image = sample_batched['image'].to(device)
        gt = sample_batched['gt'].to(device)
        '''
        inputs = Variable(image)
        print('inputs', inputs.size())

        outputs = model(image)
        optimizer.zero_grad()
        '''

        optimizer.zero_grad()
        #out, aux = model(image)
        #out = model(image)
        output, total_loss = model.eval_net_with_loss(model, image, gt, class_weights, device)
        total_loss.backward()
        optimizer.step()

        mean_loss += total_loss.cpu().detach().numpy()

        # Measure accuracy
        
        gt = np.squeeze(sample_batched['gt'].cpu().numpy())
        
        label_out = torch.nn.functional.softmax(output, dim = 1)
        label_out = label_out.cpu().detach().numpy()
        label_out = np.squeeze(label_out)
        
        labels = np.argmax(label_out, axis=0)
        valid_mask = gt != -1
        curr_correct = np.sum(gt[valid_mask] == labels[valid_mask])
        curr_false = np.sum(valid_mask) - curr_correct
        n_correct += curr_correct
        n_false += curr_false

    mean_loss /= len(train_loader)
    train_acc = n_correct / (n_correct + n_false)

    print('Train loss: %f, train acc: %f' % (mean_loss, train_acc))

    n_correct = 0
    n_false = 0

    for i_batch, sample_batched in enumerate(val_loader):

        image = sample_batched['image'].to(device)
        image_np = np.squeeze(sample_batched['image_original'].cpu().numpy())
        gt = np.squeeze(sample_batched['gt'].cpu().numpy())
        
    
        label_out = model(image)
        label_out = torch.nn.functional.softmax(label_out, dim = 1)
        label_out = label_out.cpu().detach().numpy()
        label_out = np.squeeze(label_out)

        labels = np.argmax(label_out, axis=0)

        if plot_val:
            color_label = np.zeros((resolution_input[1], resolution_input[0], 3))

            for key, val in id_to_class.items():
                color_label[labels == key] = class_to_color[val]
                
            plt.figure()
            plt.imshow((image_np/255) * 0.5 + (color_label/255) * 0.5)
            plt.show()
            
            plt.figure()
            plt.imshow(color_label.astype(np.uint8))
            plt.show()

        valid_mask = gt != -1
        curr_correct = np.sum(gt[valid_mask] == labels[valid_mask])
        curr_false = np.sum(valid_mask) - curr_correct
        n_correct += curr_correct
        n_false += curr_false

    total_acc = n_correct / (n_correct + n_false)

    if best_val_acc < total_acc:
        best_val_acc = total_acc
        if epoch > 7:
            torch.save(model.state_dict(), model_file_name)
            print('New best validation acc. Saving...')
        best_epoch = epoch

    if (epoch - best_epoch) > patience:
        print("Fnishing training, best validation acc %f", best_val_acc)
        break

    print('Val acc: %f -- Best val acc: %f -- epoch %d.' % (total_acc, best_val_acc, best_epoch))

# Inferência de dados
Processo de usar um modelo treinado para fazer previsões sobre novos dados.

In [ ]:
# Color in RGB
class_to_color = {'Ground': (127, 0, 0) , 'Healthy': (0, 127, 127), 'Pest': (0, 255, 0)}
class_to_id = {'Ground': 0, 'Healthy': 1, 'Pest': 2}
id_to_class = {v: k for k, v in class_to_id.items()}
nClasses = 3
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]


model = PSPNet(nClasses)
model.load_state_dict(torch.load(model_file_name))
model.eval()
model.to(device)

img_list = glob.glob(osp.join(img_folder_val, '*.png'))

for img_path in img_list:

        img_np = cv2.imread(img_path, cv2.IMREAD_IGNORE_ORIENTATION + cv2.IMREAD_COLOR)
        img_np = cv2.resize(img_np, (resolution_input[0], resolution_input[1]))[..., ::-1]
        img_np = np.ascontiguousarray(img_np)
        
        img_pt = np.copy(img_np).astype(np.float32) / 255.0
        for i in range(3):
            img_pt[..., i] -= mean[i]
            img_pt[..., i] /= std[i]
            
        img_pt = img_pt.transpose(2,0,1)
            
        img_pt = torch.from_numpy(img_pt[None, ...]).to(device)
        
        label_out = model(img_pt)
        label_out = torch.nn.functional.softmax(label_out, dim = 1)
        label_out = label_out.cpu().detach().numpy()
        label_out = np.squeeze(label_out)
        
        labels = np.argmax(label_out, axis=0)
        
        color_label = np.zeros((resolution_input[1], resolution_input[0], 3))
            
        for key, val in id_to_class.items():
            color_label[labels == key] = class_to_color[val]
            
        plt.figure()
        plt.imshow((img_np/255) * 0.5 + (color_label/255) * 0.5)
        plt.savefig(save_dir + "IMG" + ".png")
        plt.show()
        
        plt.figure()
        plt.imshow(color_label.astype(np.uint8))
        plt.savefig(save_dir + "GT" + ".png")
        plt.show()